# Notes

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import OrderedDict

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler,StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from imblearn.pipeline import Pipeline 
from sklearn.metrics import mean_squared_error

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd /content/drive/MyDrive/JOB-HUNTING/CV_NOW/Accenture_REPL_DS/exercise_traffic_only

/content/drive/MyDrive/JOB-HUNTING/CV_NOW/Accenture_REPL_DS/exercise_traffic_only


In [5]:
hourly_sales = pd.read_csv('data/hourly_sales_features.csv')
orig_columns = hourly_sales.columns

type conversion: convert string to datetime

In [6]:
hourly_sales.hour = pd.to_datetime(hourly_sales.hour)


In [7]:
hourly_sales.head()

,hour,num_of_hours,num_of_weeks,sales,AR__sales_weekly_total_52weeks_ago,AR__sales_weekly_total_51weeks_ago,AR__sales_1_years_ago_same_hod,AR__sales_5_weeks_ago_p_0h,AR__sales_6_weeks_ago_p_0h,seasonal__hod,seasonal__dow,holiday__is_easter,holiday__is_thanksgiving,holiday__is_thanksgiving_post_1d,holiday__is_thanksgiving_post_2d,holiday__is_thanksgiving_post_3d,holiday__is_christmas,holiday__is_christmas_post_1d,holiday__between_thanksgiving_n_christmas,holiday__days_to_next_christmas
0,2015-01-01 10:00:00,0,0,8.0,NaN,NaN,NaN,NaN,NaN,10,3,False,False,False,False,False,False,False,False,358.0
1,2015-01-01 11:00:00,1,0,14.0,NaN,NaN,NaN,NaN,NaN,11,3,False,False,False,False,False,False,False,False,358.0
2,2015-01-01 12:00:00,2,0,28.0,NaN,NaN,NaN,NaN,NaN,12,3,False,False,False,False,False,False,False,False,358.0
3,2015-01-01 13:00:00,3,0,33.0,NaN,NaN,NaN,NaN,NaN,13,3,False,False,False,False,False,False,False,False,358.0
4,2015-01-01 14:00:00,4,0,47.0,NaN,NaN,NaN,NaN,NaN,14,3,False,False,False,False,False,False,False,False,358.0


In [8]:
hourly_sales.shape

(29316, 20)

In [9]:
hourly_sales.columns

Index(['hour', 'num_of_hours', 'num_of_weeks', 'sales',
       'AR__sales_weekly_total_52weeks_ago',
       'AR__sales_weekly_total_51weeks_ago', 'AR__sales_1_years_ago_same_hod',
       'AR__sales_5_weeks_ago_p_0h', 'AR__sales_6_weeks_ago_p_0h',
       'seasonal__hod', 'seasonal__dow', 'holiday__is_easter',
       'holiday__is_thanksgiving', 'holiday__is_thanksgiving_post_1d',
       'holiday__is_thanksgiving_post_2d', 'holiday__is_thanksgiving_post_3d',
       'holiday__is_christmas', 'holiday__is_christmas_post_1d',
       'holiday__between_thanksgiving_n_christmas',
       'holiday__days_to_next_christmas'],
      dtype='object')

# Feature data transformation

remove data without full features because we use AR features 1 year ago.

In [10]:
print(hourly_sales.shape)
begin_time = hourly_sales.hour.iloc[0]
hourly_sales = hourly_sales[hourly_sales.hour > begin_time+pd.Timedelta(370,'d')]
print(hourly_sales.shape)

(29316, 20)
(20435, 20)


## keep features & labels only

In [11]:
hourly_sales.set_index('num_of_hours',inplace = True) # useful for validation and testing
hourly_sales.drop(columns = ['num_of_weeks'], axis = 1, inplace = True) # keep column 'hour' for train test split

In [12]:
hourly_sales.head()

,hour,sales,AR__sales_weekly_total_52weeks_ago,AR__sales_weekly_total_51weeks_ago,AR__sales_1_years_ago_same_hod,AR__sales_5_weeks_ago_p_0h,AR__sales_6_weeks_ago_p_0h,seasonal__hod,seasonal__dow,holiday__is_easter,holiday__is_thanksgiving,holiday__is_thanksgiving_post_1d,holiday__is_thanksgiving_post_2d,holiday__is_thanksgiving_post_3d,holiday__is_christmas,holiday__is_christmas_post_1d,holiday__between_thanksgiving_n_christmas,holiday__days_to_next_christmas
num_of_hours,,,,,,,,,,,,,,,,,,
8881,2016-01-06 11:00:00,18.0,2024.0,1393.0,23.0,19.0,15.0,11,2,False,False,False,False,False,False,False,False,354.0
8882,2016-01-06 12:00:00,23.0,2024.0,1393.0,29.0,26.0,33.0,12,2,False,False,False,False,False,False,False,False,354.0
8883,2016-01-06 13:00:00,22.0,2024.0,1393.0,19.0,22.0,34.0,13,2,False,False,False,False,False,False,False,False,354.0
8884,2016-01-06 14:00:00,28.0,2024.0,1393.0,21.0,22.0,37.0,14,2,False,False,False,False,False,False,False,False,354.0
8885,2016-01-06 15:00:00,27.0,2024.0,1393.0,15.0,16.0,51.0,15,2,False,False,False,False,False,False,False,False,354.0


## Null Value check

In [13]:
pd.set_option('display.max_rows', None)
hourly_sales.isna().sum()

hour                                         0
sales                                        0
AR__sales_weekly_total_52weeks_ago           0
AR__sales_weekly_total_51weeks_ago           0
AR__sales_1_years_ago_same_hod               0
AR__sales_5_weeks_ago_p_0h                   0
AR__sales_6_weeks_ago_p_0h                   0
seasonal__hod                                0
seasonal__dow                                0
holiday__is_easter                           0
holiday__is_thanksgiving                     0
holiday__is_thanksgiving_post_1d             0
holiday__is_thanksgiving_post_2d             0
holiday__is_thanksgiving_post_3d             0
holiday__is_christmas                        0
holiday__is_christmas_post_1d                0
holiday__between_thanksgiving_n_christmas    0
holiday__days_to_next_christmas              0
dtype: int64

## Categorical & boolean Feature Transformation

In [14]:
hourly_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20435 entries, 8881 to 29315
Data columns (total 18 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   hour                                       20435 non-null  datetime64[ns]
 1   sales                                      20435 non-null  float64       
 2   AR__sales_weekly_total_52weeks_ago         20435 non-null  float64       
 3   AR__sales_weekly_total_51weeks_ago         20435 non-null  float64       
 4   AR__sales_1_years_ago_same_hod             20435 non-null  float64       
 5   AR__sales_5_weeks_ago_p_0h                 20435 non-null  float64       
 6   AR__sales_6_weeks_ago_p_0h                 20435 non-null  float64       
 7   seasonal__hod                              20435 non-null  int64         
 8   seasonal__dow                              20435 non-null  int64         
 9   holiday__is_ea

In [15]:
# one-hot encoding for categorical features
categorical_featuers = [feature for feature in list(hourly_sales.columns) if 'seasonal' in feature]
print(categorical_featuers)
print(hourly_sales.shape)
for feature in categorical_featuers:
  hourly_sales = pd.concat(
      [hourly_sales.drop(feature,axis=1),
      pd.get_dummies(hourly_sales[feature],
                      prefix=feature+'_')],
                      axis=1)
print(hourly_sales.shape)

['seasonal__hod', 'seasonal__dow']
(20435, 18)
(20435, 47)


In [16]:
# convert booelan to int
boolean_features = [feature for feature in list(hourly_sales.columns) if 'holiday' in feature]
boolean_features.remove('holiday__days_to_next_christmas')
for feature in boolean_features:
  print(feature)
print(hourly_sales.shape)
for feature in boolean_features:
  hourly_sales[feature] = hourly_sales[feature].astype(int)
print(hourly_sales.shape)

holiday__is_easter
holiday__is_thanksgiving
holiday__is_thanksgiving_post_1d
holiday__is_thanksgiving_post_2d
holiday__is_thanksgiving_post_3d
holiday__is_christmas
holiday__is_christmas_post_1d
holiday__between_thanksgiving_n_christmas
(20435, 47)
(20435, 47)


# Split dataset to train, validation and test

Data is very valuable in this dataset, I split the dataset into only train, test sets, rather than train, dev, test sets.

In [17]:
hourly_sales.shape

(20435, 47)

In [18]:
hourly_sales.head()

,hour,sales,AR__sales_weekly_total_52weeks_ago,AR__sales_weekly_total_51weeks_ago,AR__sales_1_years_ago_same_hod,AR__sales_5_weeks_ago_p_0h,AR__sales_6_weeks_ago_p_0h,holiday__is_easter,holiday__is_thanksgiving,holiday__is_thanksgiving_post_1d,holiday__is_thanksgiving_post_2d,holiday__is_thanksgiving_post_3d,holiday__is_christmas,holiday__is_christmas_post_1d,holiday__between_thanksgiving_n_christmas,holiday__days_to_next_christmas,seasonal__hod__0,seasonal__hod__1,seasonal__hod__2,seasonal__hod__3,seasonal__hod__4,seasonal__hod__5,seasonal__hod__6,seasonal__hod__7,seasonal__hod__8,seasonal__hod__9,seasonal__hod__10,seasonal__hod__11,seasonal__hod__12,seasonal__hod__13,seasonal__hod__14,seasonal__hod__15,seasonal__hod__16,seasonal__hod__17,seasonal__hod__18,seasonal__hod__19,seasonal__hod__20,seasonal__hod__21,seasonal__hod__22,seasonal__hod__23,seasonal__dow__0,seasonal__dow__1,seasonal__dow__2,seasonal__dow__3,seasonal__dow__4,seasonal__dow__5,seasonal__dow__6
num_of_hours,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8881,2016-01-06 11:00:00,18.0,2024.0,1393.0,23.0,19.0,15.0,0,0,0,0,0,0,0,0,354.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8882,2016-01-06 12:00:00,23.0,2024.0,1393.0,29.0,26.0,33.0,0,0,0,0,0,0,0,0,354.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8883,2016-01-06 13:00:00,22.0,2024.0,1393.0,19.0,22.0,34.0,0,0,0,0,0,0,0,0,354.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8884,2016-01-06 14:00:00,28.0,2024.0,1393.0,21.0,22.0,37.0,0,0,0,0,0,0,0,0,354.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8885,2016-01-06 15:00:00,27.0,2024.0,1393.0,15.0,16.0,51.0,0,0,0,0,0,0,0,0,354.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [19]:
# remove hourly data not with full day from the last day
print(hourly_sales.hour.iloc[-1])
hourly_sales = hourly_sales[hourly_sales.hour <  pd.to_datetime(hourly_sales.hour.iloc[-1].date()) ]
print(hourly_sales.hour.iloc[-1])

2018-05-06 21:00:00
2018-05-05 23:00:00


In [20]:
total_days = (hourly_sales.hour.iloc[-1] - hourly_sales.hour.iloc[0]).days
print('total_days: ', total_days)
print('15% days: ', int(total_days * 0.15))
test_days = 180
train_days = total_days - test_days
print('used train_days: ', train_days)
# print('used dev_days: ', dev_days)
print('used test_days: ', test_days)

total_days:  850
15% days:  127
used train_days:  670
used test_days:  180


In [21]:
train_data = hourly_sales[hourly_sales.hour <= hourly_sales.hour.iloc[-1]-pd.Timedelta(test_days,'d') ]
test_data = hourly_sales[hourly_sales.hour > hourly_sales.hour.iloc[-1]-pd.Timedelta(test_days,'d' )]
print('train data range')
print(train_data.hour.iloc[0])
print(train_data.hour.iloc[-1])
print()
print('test data range')
print(test_data.hour.iloc[0])
print(test_data.hour.iloc[-1])

train data range
2016-01-06 11:00:00
2017-11-06 23:00:00

test data range
2017-11-07 00:00:00
2018-05-05 23:00:00


In [22]:
print('test_date has {} days'.format(test_data.shape[0]/24))

test_date has 180.0 days


## Adding predicted sales as features

let's add predicted sales as features for predicting traffic



In [23]:
print('train data range')
print(train_data.hour.iloc[0])
print(train_data.hour.iloc[-1])
print()
print('test data range')
print(test_data.hour.iloc[0])
print(test_data.hour.iloc[-1])

train data range
2016-01-06 11:00:00
2017-11-06 23:00:00

test data range
2017-11-07 00:00:00
2018-05-05 23:00:00


In [24]:
# adding sales data to traffic data as one of input features
train_data_sales = pd.read_csv('data/train_data_sales.csv')
train_data_sales.hour = pd.to_datetime(train_data_sales.hour)
train_data_sales = train_data_sales[['hour','sales']]
train_data_sales.rename(columns = {'sales':'sales_feature'},inplace = True)
train_data = train_data.merge(train_data_sales, how = 'left', on = 'hour')
print(train_data.shape)
train_data.head()

(16093, 48)


,hour,sales,AR__sales_weekly_total_52weeks_ago,AR__sales_weekly_total_51weeks_ago,AR__sales_1_years_ago_same_hod,AR__sales_5_weeks_ago_p_0h,AR__sales_6_weeks_ago_p_0h,holiday__is_easter,holiday__is_thanksgiving,holiday__is_thanksgiving_post_1d,holiday__is_thanksgiving_post_2d,holiday__is_thanksgiving_post_3d,holiday__is_christmas,holiday__is_christmas_post_1d,holiday__between_thanksgiving_n_christmas,holiday__days_to_next_christmas,seasonal__hod__0,seasonal__hod__1,seasonal__hod__2,seasonal__hod__3,seasonal__hod__4,seasonal__hod__5,seasonal__hod__6,seasonal__hod__7,seasonal__hod__8,seasonal__hod__9,seasonal__hod__10,seasonal__hod__11,seasonal__hod__12,seasonal__hod__13,seasonal__hod__14,seasonal__hod__15,seasonal__hod__16,seasonal__hod__17,seasonal__hod__18,seasonal__hod__19,seasonal__hod__20,seasonal__hod__21,seasonal__hod__22,seasonal__hod__23,seasonal__dow__0,seasonal__dow__1,seasonal__dow__2,seasonal__dow__3,seasonal__dow__4,seasonal__dow__5,seasonal__dow__6,sales_feature
0,2016-01-06 11:00:00,18.0,2024.0,1393.0,23.0,19.0,15.0,0,0,0,0,0,0,0,0,354.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,3504.0
1,2016-01-06 12:00:00,23.0,2024.0,1393.0,29.0,26.0,33.0,0,0,0,0,0,0,0,0,354.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,3547.0
2,2016-01-06 13:00:00,22.0,2024.0,1393.0,19.0,22.0,34.0,0,0,0,0,0,0,0,0,354.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,3445.0
3,2016-01-06 14:00:00,28.0,2024.0,1393.0,21.0,22.0,37.0,0,0,0,0,0,0,0,0,354.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,3419.0
4,2016-01-06 15:00:00,27.0,2024.0,1393.0,15.0,16.0,51.0,0,0,0,0,0,0,0,0,354.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,3327.0


In [25]:
test_data_sales = pd.read_csv('data/test_data_sales.csv')
test_data_sales.hour = pd.to_datetime(test_data_sales.hour)
test_data_sales = test_data_sales[['hour','predicted_sales']]
test_data_sales.rename(columns = {'predicted_sales':'sales_feature'},inplace = True)
test_data = test_data.merge(test_data_sales, how = 'left', on = 'hour')
print(test_data.shape)
test_data.head()

(4320, 48)


,hour,sales,AR__sales_weekly_total_52weeks_ago,AR__sales_weekly_total_51weeks_ago,AR__sales_1_years_ago_same_hod,AR__sales_5_weeks_ago_p_0h,AR__sales_6_weeks_ago_p_0h,holiday__is_easter,holiday__is_thanksgiving,holiday__is_thanksgiving_post_1d,holiday__is_thanksgiving_post_2d,holiday__is_thanksgiving_post_3d,holiday__is_christmas,holiday__is_christmas_post_1d,holiday__between_thanksgiving_n_christmas,holiday__days_to_next_christmas,seasonal__hod__0,seasonal__hod__1,seasonal__hod__2,seasonal__hod__3,seasonal__hod__4,seasonal__hod__5,seasonal__hod__6,seasonal__hod__7,seasonal__hod__8,seasonal__hod__9,seasonal__hod__10,seasonal__hod__11,seasonal__hod__12,seasonal__hod__13,seasonal__hod__14,seasonal__hod__15,seasonal__hod__16,seasonal__hod__17,seasonal__hod__18,seasonal__hod__19,seasonal__hod__20,seasonal__hod__21,seasonal__hod__22,seasonal__hod__23,seasonal__dow__0,seasonal__dow__1,seasonal__dow__2,seasonal__dow__3,seasonal__dow__4,seasonal__dow__5,seasonal__dow__6,sales_feature
0,2017-11-07 00:00:00,0.0,1015.0,1218.0,0.0,0.0,1.0,0,0,0,0,0,0,0,0,48.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.0
1,2017-11-07 01:00:00,0.0,1015.0,1218.0,1.0,0.0,0.0,0,0,0,0,0,0,0,0,48.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.0
2,2017-11-07 02:00:00,0.0,1015.0,1218.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,48.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.0
3,2017-11-07 03:00:00,1.0,1015.0,1218.0,0.0,1.0,0.0,0,0,0,0,0,0,0,0,48.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.0
4,2017-11-07 04:00:00,0.0,1015.0,1218.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,48.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.0


## seperate features and labels

In [26]:
train_data_x = train_data.drop(columns = ['hour','sales'], axis = 1)
train_data_y = train_data['sales']

In [27]:
test_data_x = test_data.drop(columns = ['hour','sales'], axis = 1)
test_data_y = test_data['sales']

In [28]:
print(train_data_x.shape)
print(train_data_y.shape)
print(test_data_x.shape)
print(test_data_y.shape)


(16093, 46)
(16093,)
(4320, 46)
(4320,)


# Model Selection

The following models are trained: logistic regression, random froest. If we want to model trend with non-linear models, probably neural network is better.

In [29]:
performance = pd.DataFrame({'train':[None]*2,'test': [None]*2, 'test_real_time': [None]*2})
performance.index = ['LR', 'RF']
performance

,train,test,test_real_time
LR,None,None,None
RF,None,None,None


In [30]:
hourly_sales.head()

,hour,sales,AR__sales_weekly_total_52weeks_ago,AR__sales_weekly_total_51weeks_ago,AR__sales_1_years_ago_same_hod,AR__sales_5_weeks_ago_p_0h,AR__sales_6_weeks_ago_p_0h,holiday__is_easter,holiday__is_thanksgiving,holiday__is_thanksgiving_post_1d,holiday__is_thanksgiving_post_2d,holiday__is_thanksgiving_post_3d,holiday__is_christmas,holiday__is_christmas_post_1d,holiday__between_thanksgiving_n_christmas,holiday__days_to_next_christmas,seasonal__hod__0,seasonal__hod__1,seasonal__hod__2,seasonal__hod__3,seasonal__hod__4,seasonal__hod__5,seasonal__hod__6,seasonal__hod__7,seasonal__hod__8,seasonal__hod__9,seasonal__hod__10,seasonal__hod__11,seasonal__hod__12,seasonal__hod__13,seasonal__hod__14,seasonal__hod__15,seasonal__hod__16,seasonal__hod__17,seasonal__hod__18,seasonal__hod__19,seasonal__hod__20,seasonal__hod__21,seasonal__hod__22,seasonal__hod__23,seasonal__dow__0,seasonal__dow__1,seasonal__dow__2,seasonal__dow__3,seasonal__dow__4,seasonal__dow__5,seasonal__dow__6
num_of_hours,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8881,2016-01-06 11:00:00,18.0,2024.0,1393.0,23.0,19.0,15.0,0,0,0,0,0,0,0,0,354.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8882,2016-01-06 12:00:00,23.0,2024.0,1393.0,29.0,26.0,33.0,0,0,0,0,0,0,0,0,354.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8883,2016-01-06 13:00:00,22.0,2024.0,1393.0,19.0,22.0,34.0,0,0,0,0,0,0,0,0,354.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8884,2016-01-06 14:00:00,28.0,2024.0,1393.0,21.0,22.0,37.0,0,0,0,0,0,0,0,0,354.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8885,2016-01-06 15:00:00,27.0,2024.0,1393.0,15.0,16.0,51.0,0,0,0,0,0,0,0,0,354.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


## Training set performance

In [31]:
train_data_x.columns

Index(['AR__sales_weekly_total_52weeks_ago',
       'AR__sales_weekly_total_51weeks_ago', 'AR__sales_1_years_ago_same_hod',
       'AR__sales_5_weeks_ago_p_0h', 'AR__sales_6_weeks_ago_p_0h',
       'holiday__is_easter', 'holiday__is_thanksgiving',
       'holiday__is_thanksgiving_post_1d', 'holiday__is_thanksgiving_post_2d',
       'holiday__is_thanksgiving_post_3d', 'holiday__is_christmas',
       'holiday__is_christmas_post_1d',
       'holiday__between_thanksgiving_n_christmas',
       'holiday__days_to_next_christmas', 'seasonal__hod__0',
       'seasonal__hod__1', 'seasonal__hod__2', 'seasonal__hod__3',
       'seasonal__hod__4', 'seasonal__hod__5', 'seasonal__hod__6',
       'seasonal__hod__7', 'seasonal__hod__8', 'seasonal__hod__9',
       'seasonal__hod__10', 'seasonal__hod__11', 'seasonal__hod__12',
       'seasonal__hod__13', 'seasonal__hod__14', 'seasonal__hod__15',
       'seasonal__hod__16', 'seasonal__hod__17', 'seasonal__hod__18',
       'seasonal__hod__19', 'seasonal__h

In [32]:
regressors = [
    LinearRegression(),
    # DecisionTreeClassifier(),
    RandomForestRegressor()
    # GradientBoostingRegressor(),
    ]

Train model using training data

In [33]:
scaler = StandardScaler()
scaler.fit(train_data_x)
train_data_x_scaled = scaler.transform(train_data_x)

In [34]:
# def build_ml_pipeline(regressor):
#   steps = []
#   steps.append(('fillnan', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0.0)))
#   steps.append(('scaler',StandardScaler())) #scaler applies only to training data, not validation data to avoid data leakage
#   steps.append(('model', regressor))
#   pipeline = Pipeline(steps = steps)
#   return pipeline

### linear regression

In [35]:
#-----linear regression
regressor = LinearRegression()
# training model using train_set
regressor.fit(train_data_x_scaled,train_data_y)
# performance on training set
y_predicted = regressor.predict(train_data_x_scaled)
rmse = mean_squared_error(train_data_y, y_predicted, squared=False) # RMS
print(regressor.__class__.__name__, ':RMSE is (%.3f)'%rmse)
linear_regressor = regressor
performance.loc['LR','train'] = rmse

LinearRegression :RMSE is (4.865)


In [36]:
# fig, ax = plt.subplots()
# markerline1, stemlines1, baseline = plt.stem(range(len(train_data_y)),train_data_y, 'b',markerfmt = 'bo', label = 'truth')
# plt.setp(stemlines1, 'linewidth', 0.5,'markerfacecolor', 'b')

# markerline2, stemlines2, baseline = plt.stem(range(len(y_predicted)), y_predicted, 'r-.',markerfmt = 'rx', label = 'predicted')
# plt.setp(stemlines2, 'linewidth', 0.5,'markerfacecolor', 'r')
# fig = plt.gcf()
# fig.set_size_inches(46, 9)
# plt.xlim([-1,100])
# plt.show()

### Random forest

In [37]:
#-----RF regression
regressor = RandomForestRegressor() # GradientBoostingRegressor RandomForestRegressor
# training model using train_set
regressor.fit(train_data_x_scaled,train_data_y)
# performance on training set
y_predicted = regressor.predict(train_data_x_scaled)
rmse = mean_squared_error(train_data_y, y_predicted, squared=False) # RMS
print(regressor.__class__.__name__, ':RMSE is (%.3f)'%rmse)
RF_regressor = regressor
performance.loc['RF','train'] = rmse

RandomForestRegressor :RMSE is (1.288)


In [38]:
# fig, ax = plt.subplots()
# markerline1, stemlines1, baseline = plt.stem(range(len(train_data_y)),train_data_y, 'b',markerfmt = 'bo', label = 'truth')
# plt.setp(stemlines1, 'linewidth', 0.5,'markerfacecolor', 'b')

# markerline2, stemlines2, baseline = plt.stem(range(len(y_predicted)), y_predicted, 'r-.',markerfmt = 'rx', label = 'predicted')
# plt.setp(stemlines2, 'linewidth', 0.5,'markerfacecolor', 'r')
# fig = plt.gcf()
# fig.set_size_inches(46, 9)
# plt.xlim([-1,100])
# plt.show()

### Feature imoprtance

#### Importance LR

In [39]:
import math
coeff = np.squeeze(linear_regressor.coef_)
importance = abs(np.array(coeff))/np.sum(abs(np.array(coeff)))*100
sort_index = np.argsort(importance)
data = {'features':np.array(train_data_x.columns)[sort_index[::-1]], 'importance':importance[sort_index[::-1]] }
pd.DataFrame(data)

,features,importance
0,seasonal__dow__6,3.435360e+00
1,seasonal__dow__5,3.435360e+00
2,seasonal__dow__4,3.435360e+00
3,seasonal__dow__0,3.435360e+00
4,seasonal__dow__3,3.435360e+00
5,seasonal__dow__2,3.428517e+00
6,seasonal__dow__1,3.420394e+00
7,seasonal__hod__11,3.166630e+00
8,seasonal__hod__12,3.166630e+00
9,seasonal__hod__20,3.166630e+00


#### Importance RF

In [40]:
importances = RF_regressor.feature_importances_
feature_names = train_data_x.columns.tolist()

In [41]:
df_importances = pd.DataFrame(
    {"feature":feature_names, "importance":importances}).sort_values("importance", ascending=False)
# top15_features = df_importances   
# top15_features
df_importances

,feature,importance
2,AR__sales_1_years_ago_same_hod,0.665297
45,sales_feature,0.222043
13,holiday__days_to_next_christmas,0.032835
4,AR__sales_6_weeks_ago_p_0h,0.019026
3,AR__sales_5_weeks_ago_p_0h,0.014382
0,AR__sales_weekly_total_52weeks_ago,0.008824
1,AR__sales_weekly_total_51weeks_ago,0.008077
7,holiday__is_thanksgiving_post_1d,0.004817
42,seasonal__dow__4,0.002038
38,seasonal__dow__0,0.002034


## Test set performance

### Random Forest

In [42]:
test_data_x_scaled = scaler.transform(test_data_x)

In [43]:
#-----linear regression
# performance on training set
y_predicted = linear_regressor.predict(test_data_x_scaled)
rmse = mean_squared_error(test_data_y, y_predicted, squared=False) # RMS
print(linear_regressor.__class__.__name__, ':RMSE is (%.3f)'%rmse)
performance.loc['LR','test'] = rmse

LinearRegression :RMSE is (6.306)


### Random Froest

In [44]:
#-----RF
y_predicted = RF_regressor.predict(test_data_x_scaled)
rmse = mean_squared_error(test_data_y, y_predicted, squared=False) # RMS
print(RF_regressor.__class__.__name__, ':RMSE is (%.3f)'%rmse)
performance.loc['RF','test'] = rmse

RandomForestRegressor :RMSE is (4.896)


## Hyper parameter tuning

### Random Froest

In [45]:
# n_estimators_vector = [50,100,150]# [50,100,150]
# min_samples_leaf_vector = [1,2,3] #[1,2,3]
# min_samples_split_vector = [2,3,4] #[2,3,4]
# n1,n2,n3 = len(n_estimators_vector), len(min_samples_leaf_vector), len(min_samples_split_vector)
# RF_performance = pd.DataFrame(index = range(n1*n2*n3), columns = ['n_estimators','min_samples_leaf','min_samples_split','train_rmse','test_rmse'])

# i  = 0
# for n_estimators in n_estimators_vector:
#   for min_samples_leaf in min_samples_leaf_vector:
#     for min_samples_split in min_samples_split_vector:
#       print(i)
#       #-----training
#       regressor = RandomForestRegressor(n_estimators = n_estimators, min_samples_leaf = min_samples_leaf,min_samples_split = min_samples_split)
#       regressor.fit(train_data_x_scaled,train_data_y)
#       y_predicted = regressor.predict(train_data_x_scaled)
#       train_rmse = mean_squared_error(train_data_y, y_predicted, squared=False) # RMS
#       #-----testing
#       y_predicted = regressor.predict(test_data_x_scaled)
#       test_rmse = mean_squared_error(test_data_y, y_predicted, squared=False) # RMS

#       RF_performance.iloc[i,:] = [n_estimators,min_samples_leaf,min_samples_split,train_rmse,test_rmse]
#       i += 1
# RF_performance.sort_values('test_rmse')

## Test set real-time performance

In this section, the performance is evaluated on the data chunk of every 30 days. This is to evaluate the actual performance when predicting hourly sales/traffic for the next month. 

 The following auto regressive features' values needs to be updated in real-time manner, meaning that when a sale value for certain time(hour) is predicted. The newly predicted sale value should be used as features rather than using the truth data. This is because we need to predict sales 30 days ahead. 

In [46]:
update_features = [ 'AR__sales_1_weeks_ago_m_1h',
                    'AR__sales_1_weeks_ago_p_0h',
                    'AR__sales_1_weeks_ago_p_1h',
                    'AR__sales_2_weeks_ago_m_1h',
                    'AR__sales_2_weeks_ago_p_0h',
                    'AR__sales_2_weeks_ago_p_1h',
                    'AR__sales_3_weeks_ago_m_1h',
                    'AR__sales_3_weeks_ago_p_0h',
                    'AR__sales_3_weeks_ago_p_1h',
                    'AR__sales_4_weeks_ago_m_1h',
                    'AR__sales_4_weeks_ago_p_0h',
                    'AR__sales_4_weeks_ago_p_1h',
                    'AR__sales_1_days_ago_m_1h',
                    'AR__sales_1_days_ago_p_0h',
                    'AR__sales_1_days_ago_p_1h',
                    'AR__sales_2_days_ago_m_1h',
                    'AR__sales_2_days_ago_p_0h',
                    'AR__sales_2_days_ago_p_1h',
                    'AR__sales_3_days_ago_m_1h',
                    'AR__sales_3_days_ago_p_0h',
                    'AR__sales_3_days_ago_p_1h',
                    'AR__sales_4_days_ago_m_1h',
                    'AR__sales_4_days_ago_p_0h',
                    'AR__sales_4_days_ago_p_1h',
                    'AR__sales_pre_1_h',
                    'AR__sales_pre_2_h',
                    'AR__sales_pre_3_h',
                    'AR__sales_pre_4_h',
                    'AR__sales_pre_5_h',
                    'AR__sales_pre_6_h',
                    'AR__sales_pre_7_h',
                    'AR__sales_pre_8_h',
                    'AR__sales_pre_9_h',
                    'AR__sales_pre_10_h',
                    'AR__sales_pre_11_h',
                    'AR__sales_pre_12_h',
                    'AR__sales_pre_13_h',
                    'AR__sales_pre_14_h',
                    'AR__sales_pre_15_h',
                    'AR__sales_pre_16_h',
                    'AR__sales_pre_17_h',
                    'AR__sales_pre_18_h',
                    'AR__sales_pre_19_h',
                    'AR__sales_pre_20_h',
                    'AR__sales_pre_21_h',
                    'AR__sales_pre_22_h']
update_features_copy = update_features.copy()
update_features = []
for feature in update_features_copy:
  if feature in test_data_x.columns:
    update_features.append(feature)
update_features

[]

In [47]:
update_features

[]

### linear regression

In [48]:
# number of months in test data
num_months_test = int(test_data_y.shape[0]/24/30)

we will udpate the table 'aux_hourly_sales' in real time when evaluating the model performance, note we need to copy the dataframe again for every 30 days to mimic the real practice

In [49]:
# linear regression

from utility import calc_row_index_hourly
y_predicted = np.empty((test_data_y.shape[0],),dtype = float)

for j in range(num_months_test): # iterate over each 30 days
  aux_hourly_sales = hourly_sales.copy(deep = True)
  # for i in range(test_data_x.shape[0]):
  for i in range(j*30*24, (j+1)*30*24): # iterate over jth month(30 days)
    # update AR features subject to change from possibly newly predicted data, because this problem is for predicting 1 month ahead
    hour_index = test_data_x.index[i] # hour index for current sales to be predicted
    features_cur_row = test_data_x.iloc[i,:].copy(deep = True)
    for feature in update_features:
      features_cur_row[feature] = aux_hourly_sales.loc[hour_index - calc_row_index_hourly(feature), 'sales']
    # scale data 
    scaled_features_cur_row = scaler.transform(features_cur_row.values.reshape(1,-1))
    # inference 
    y_predicted[i] = linear_regressor.predict(scaled_features_cur_row)
    # update sale value in real time
    aux_hourly_sales.loc[hour_index,'sales'] = y_predicted[i]

rmse = mean_squared_error(test_data_y, y_predicted, squared=False) # RMS
print(linear_regressor.__class__.__name__, ':RMSE is (%.3f)'%rmse)
performance.loc['LR','test_real_time'] = rmse

LinearRegression :RMSE is (6.305)


In [50]:
# fig, ax = plt.subplots()
# markerline1, stemlines1, baseline = plt.stem(range(len(test_data_y)),test_data_y, 'b',markerfmt = 'bo', label = 'truth')
# plt.setp(stemlines1, 'linewidth', 0.5,'markerfacecolor', 'b')

# markerline2, stemlines2, baseline = plt.stem(range(len(y_predicted)), y_predicted, 'r-.',markerfmt = 'rx', label = 'predicted')
# plt.setp(stemlines2, 'linewidth', 0.5,'markerfacecolor', 'r')
# fig = plt.gcf()
# fig.set_size_inches(46, 9)
# plt.xlim([-1,100])
# plt.show()

### Random forest

In [51]:
from utility import calc_row_index_hourly
y_predicted = np.empty((test_data_y.shape[0],),dtype = float)

for j in range(num_months_test): # iterate over each 30 days
  aux_hourly_sales = hourly_sales.copy(deep = True)
  # for i in range(test_data_x.shape[0]):
  for i in range(j*30*24, (j+1)*30*24): # iterate over jth month(30 days)
    # update AR features subject to change from possibly newly predicted data, because this problem is for predicting 1 month ahead
    hour_index = test_data_x.index[i] # hour index for current sales to be predicted
    features_cur_row = test_data_x.iloc[i,:].copy(deep = True)
    for feature in update_features:
      features_cur_row[feature] = aux_hourly_sales.loc[hour_index - calc_row_index_hourly(feature), 'sales']
    # scale data 
    scaled_features_cur_row = scaler.transform(features_cur_row.values.reshape(1,-1))
    # inference 
    y_predicted[i] = RF_regressor.predict(scaled_features_cur_row)
    # update sale value in real time
    aux_hourly_sales.loc[hour_index,'sales'] = y_predicted[i]

rmse = mean_squared_error(test_data_y, y_predicted, squared=False) # RMS
print(RF_regressor.__class__.__name__, ':RMSE is (%.3f)'%rmse)
performance.loc['RF','test_real_time'] = rmse

RandomForestRegressor :RMSE is (4.896)


In [52]:
# fig, ax = plt.subplots()
# markerline1, stemlines1, baseline = plt.stem(range(len(test_data_y)),test_data_y, 'b',markerfmt = 'bo', label = 'truth')
# plt.setp(stemlines1, 'linewidth', 0.5,'markerfacecolor', 'b')

# markerline2, stemlines2, baseline = plt.stem(range(len(y_predicted)), y_predicted, 'r-.',markerfmt = 'rx', label = 'predicted')
# plt.setp(stemlines2, 'linewidth', 0.5,'markerfacecolor', 'r')
# fig = plt.gcf()
# fig.set_size_inches(46, 9)
# plt.xlim([350,750])
# plt.show()

# Performance summary table

In [53]:
orig_columns

Index(['hour', 'num_of_hours', 'num_of_weeks', 'sales',
       'AR__sales_weekly_total_52weeks_ago',
       'AR__sales_weekly_total_51weeks_ago', 'AR__sales_1_years_ago_same_hod',
       'AR__sales_5_weeks_ago_p_0h', 'AR__sales_6_weeks_ago_p_0h',
       'seasonal__hod', 'seasonal__dow', 'holiday__is_easter',
       'holiday__is_thanksgiving', 'holiday__is_thanksgiving_post_1d',
       'holiday__is_thanksgiving_post_2d', 'holiday__is_thanksgiving_post_3d',
       'holiday__is_christmas', 'holiday__is_christmas_post_1d',
       'holiday__between_thanksgiving_n_christmas',
       'holiday__days_to_next_christmas'],
      dtype='object')

The performance with predicted sales as a feature is better than that without the predicted sale. This could be because of 2 reasons:
1. sales does provide more information and therefore a good predictor for traffic. In this case, we probably should use AR sales data as features.
2. We have more sales data than traffic data, the trained sales prediction model with more training data could be better than trained traffic model with less training model.

We can actually also do the same thing when predicting sales, meaning that we train traffic model first, then use predicted traffic as a feature for predicting sales. Or we can ust use both AR features from both sales and traffic when predicting sales.

In [55]:
y_predicted_corrected = y_predicted
y_predicted_corrected[y_predicted<0] = 0

rmse = mean_squared_error(test_data_y, y_predicted_corrected, squared=False) # RMS
print(regressor.__class__.__name__, ':RMSE is (%.3f)'%rmse)

RandomForestRegressor :RMSE is (4.896)


In [56]:
result = pd.DataFrame({'predicted':y_predicted,'truth': test_data_y})
result['hour'] = test_data.hour
result.to_csv('data/result.csv')

In [57]:
result.head()

,predicted,truth,hour
0,0.58,0.0,2017-11-07 00:00:00
1,0.88,0.0,2017-11-07 01:00:00
2,0.04,0.0,2017-11-07 02:00:00
3,0.14,1.0,2017-11-07 03:00:00
4,0.19,0.0,2017-11-07 04:00:00


In [54]:
performance

,train,test,test_real_time
LR,4.865198,6.305594,6.305386
RF,1.287987,4.896193,4.896193
